In [147]:
import pandas as pd
import numpy as np

# === Step 1: Load the CSV ===
#df = pd.read_csv("all_enrichments_of_STAT1_gene_and_its_neighbor_genes.csv")  # Replace with your actual CSV file path
#df = pd.read_csv("all_enrichments_of_SNX2_gene_and_its_neighbor_genes.csv")
#df = pd.read_csv("all_enrichments_of_SLC2A14_gene_and_its_neighbor_genes.csv")
#df = pd.read_csv("all_enrichments_of_LTBP4_gene_and_its_neighbor_genes.csv")
#df = pd.read_csv("all_enrichments_of_DUOX1_gene_and_its_neighbor_genes.csv")
#df = pd.read_csv("all_enrichments_of_SULF2_gene_and_its_neighbor_genes.csv")
df = pd.read_csv("all_enrichments_of_TUBGCP2_gene_and_its_neighbor_genes.csv")

#df = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=True)


In [149]:
# === Step 2: Filter rows where category is "COMPARTMENTS" ===
df_filtered = df[df['category'] == 'Function'].copy()

print(df_filtered.head())

     category        term  number_of_genes  number_of_genes_in_background  \
547  Function  GO:0015631               69                            380   
548  Function  GO:0005200               36                            107   
549  Function  GO:1901265              180                           2169   
550  Function  GO:0000166              179                           2168   
551  Function  GO:0008017               51                            269   

     ncbiTaxonId                                         inputGenes  \
547         9606  9606.ENSP00000160827,9606.ENSP00000233121,9606...   
548         9606  9606.ENSP00000217133,9606.ENSP00000247843,9606...   
549         9606  9606.ENSP00000160827,9606.ENSP00000202967,9606...   
550         9606  9606.ENSP00000160827,9606.ENSP00000202967,9606...   
551         9606  9606.ENSP00000160827,9606.ENSP00000233121,9606...   

                                        preferredNames       p_value  \
547  KIF22,MAPRE3,CAMSAP2,TTLL2,TUBGCP

In [149]:
# === Step 2: Filter rows where category is "COMPARTMENTS" ===
df_filtered = df[df['category'] == 'Function'].copy()

print(df_filtered.head())

     category        term  number_of_genes  number_of_genes_in_background  \
547  Function  GO:0015631               69                            380   
548  Function  GO:0005200               36                            107   
549  Function  GO:1901265              180                           2169   
550  Function  GO:0000166              179                           2168   
551  Function  GO:0008017               51                            269   

     ncbiTaxonId                                         inputGenes  \
547         9606  9606.ENSP00000160827,9606.ENSP00000233121,9606...   
548         9606  9606.ENSP00000217133,9606.ENSP00000247843,9606...   
549         9606  9606.ENSP00000160827,9606.ENSP00000202967,9606...   
550         9606  9606.ENSP00000160827,9606.ENSP00000202967,9606...   
551         9606  9606.ENSP00000160827,9606.ENSP00000233121,9606...   

                                        preferredNames       p_value  \
547  KIF22,MAPRE3,CAMSAP2,TTLL2,TUBGCP

In [151]:
# === Step 3: Split 'preferredNames' column into separate gene columns ===
split_genes = df_filtered['preferredNames'].str.split(',', expand=True)

# Rename columns as C1, C2, C3, ...
split_genes.columns = [f"C{i+1}" for i in range(split_genes.shape[1])]

# === Step 4: Keep only 'description' column + gene columns ===
final_df = pd.concat([df_filtered[['description']].reset_index(drop=True), split_genes], axis=1)

# === Step 5: Save to CSV (optional) ===
# final_df.to_csv("filtered_compartments_genes.csv", index=False)

# === Step 6: Show preview ===
print(final_df.head())


                              description   C1   C2   C3   C4   C5   C6   C7  \
0                         Tubulin binding  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1  Structural constituent of cytoskeleton  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2            Nucleoside phosphate binding  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3                      Nucleotide binding  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4                     Microtubule binding  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

    C8   C9  ... C595 C596 C597 C598 C599 C600 C601 C602 C603 C604  
0  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 605 columns]


In [153]:
# Example: replace with your actual gene list
gene_list = {"LTBP4", "SNX2", "STAT1", "SULF2", "SLC2A14", "TUBGCP2", "DUOX1"}  # use a set for fast lookup

# === Step 3: Check each cell for match ===
# Create a copy to store matched genes or NaN
matched_df = final_df.copy()

In [155]:
# Apply function to each cell
for col in matched_df.columns[1:]:  # skip 'Functions' column
    matched_df[col] = matched_df[col].apply(lambda x: x if pd.notna(x) and x in gene_list else np.nan)

# === Step 4: Add column for each row's matched genes ===
matched_df['Matched_Genes'] = matched_df.iloc[:, 1:].apply(lambda row: ','.join(row.dropna().astype(str)) if row.dropna().any() else np.nan, axis=1)
print(matched_df.head())
subset_df = matched_df[['description', 'Matched_Genes']]
# Remove rows where 'Matched_Genes' is NaN or an empty string
subset_df1 = subset_df[subset_df['Matched_Genes'].notna() & (subset_df['Matched_Genes'] != '')]

print(subset_df.head())
# === Step 5: Save to file ===
#subset_df.to_csv("STAT1_Process_matched_subset.csv", index=False)
subset_df1.to_csv("TUBGCP2_Function_matched_subset1.csv", index=False)

                              description  C1  C2  C3  C4  C5  C6  C7  C8  C9  \
0                         Tubulin binding NaN NaN NaN NaN NaN NaN NaN NaN NaN   
1  Structural constituent of cytoskeleton NaN NaN NaN NaN NaN NaN NaN NaN NaN   
2            Nucleoside phosphate binding NaN NaN NaN NaN NaN NaN NaN NaN NaN   
3                      Nucleotide binding NaN NaN NaN NaN NaN NaN NaN NaN NaN   
4                     Microtubule binding NaN NaN NaN NaN NaN NaN NaN NaN NaN   

   ... C596  C597  C598  C599  C600  C601  C602  C603  C604  Matched_Genes  
0  ...  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN            NaN  
1  ...  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN            NaN  
2  ...  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN            NaN  
3  ...  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN            NaN  
4  ...  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN            NaN  

[5 rows x 606 columns]
                           

C:\Users\Kabir Hossen\AppData\Local\Temp\ipykernel_32376\2034335318.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  matched_df['Matched_Genes'] = matched_df.iloc[:, 1:].apply(lambda row: ','.join(row.dropna().astype(str)) if row.dropna().any() else np.nan, axis=1)
